<a href="https://colab.research.google.com/gist/gheesung/eb0076e040ba53d5be2ad2db1c70cf82/image-classification-with-sipeed-maix-using-mobilenetv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classificazione immagini di fiori con Sipeed AIoT.

Per l'esempio in questione è stato scelto di utilizzare la rete Mobilenet V1 adattandola a una classificazione a 5 classi che sono appunto i tipi di fiori da classificare.
Il modello sarà modificato agigungendo ulteriori strati e/o tagliando alcuni strati gia esistenti per la rete MobileNet.

Clono il tool che ci permetterà di convertire il modello addestrato per l'architettura di K210 da Github e il transfer learning della MobileNet.


In [ ]:
%cd /content
! rm -rf *

/content


In [1]:
!git clone https://github.com/AIWintermuteAI/transfer_learning_sipeed.git
!git clone https://github.com/sipeed/Maix_Toolbox.git

Cloning into 'transfer_learning_sipeed'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 63
Unpacking objects: 100% (66/66), done.
Cloning into 'Maix_Toolbox'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 34 (delta 6), reused 5 (delta 5), pack-reused 25
Unpacking objects: 100% (34/34), done.


###Installo il convertitore tflite/kmodel

In [ ]:
%%bash
cd Maix_Toolbox 
mkdir -p ncc
mkdir -p workspace
mkdir -p images
mkdir -p log
cd ncc
wget https://github.com/kendryte/nncase/releases/download/v0.1.0-rc5/ncc-linux-x86_64.tar.xz
tar -Jxf ncc-linux-x86_64.tar.xz
rm ncc-linux-x86_64.tar.xz
echo "download nncase ok!"

## importo ulteriori librerie

In [3]:
import keras
import numpy as np
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D, Dropout,Flatten


import sys
sys.path.append('/content/transfer_learning_sipeed')

#from mobilenet_sipeed.mobilenet import MobileNet



from keras.applications.mobilenet import preprocess_input,MobileNet






### Scarico il dataset dei fiori 

In [4]:
%cd /content
!curl -LO http://download.tensorflow.org/example_images/flower_photos.tgz
!tar xzf flower_photos.tgz

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0   184M      0  0:00:01  0:00:01 --:--:--  184M


### Definisco i prametri per l'addestramento

In [5]:
IMAGE_SIZE = 224
ALPHA=0.75
EPOCHS=10

In [6]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [7]:
# function to define dropout, hidden layers and the number of output
# funzione per definire dropout, hidden layer e il numero di output
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

### Transfer Learning di Mobilenet V1

In [8]:
import keras
import numpy as np
from keras import backend as K
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
import sys
sys.path.append('./transfer_learning_sipeed')
from mobilenet_sipeed.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
import tensorflow

In [9]:
import tensorflow.keras as keras
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

# MobileNetv1
base_model=MobileNet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), alpha = ALPHA, 
                     depth_multiplier = 1, dropout = 0.001, include_top = False, 
                     weights = "imagenet", classes = 1000, backend=keras.backend, 
                     layers=keras.layers,models=keras.models,utils=keras.utils)


10641408/10626956 [==============================] - 1s 0us/step


In [ ]:
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

### Definisco gli ultimi strati
I used 2 hidden layers and 100 and 50 nodes. More layers or nodes can be added but this will increase the model size and may not fit into Maixpy memory.

In [11]:
FC_LAYERS = [100, 50]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=5)

In [14]:
for i,layer in enumerate(finetune_model.layers):
    print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [13]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/flower_photos',
                                                 target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical', shuffle=True)

Found 3670 images belonging to 5 classes.


### Train del Modello

In [15]:
finetune_model.summary()
finetune_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//train_generator.batch_size
history = finetune_model.fit_generator(generator=train_generator,steps_per_epoch=step_size_train,epochs=EPOCHS, shuffle=True)

finetune_model.save('/content/my_model.h5')



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 226, 226, 3)       0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 24)      648       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 24)     96        
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 24)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 24)     216       
                                                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/10
114/114 [==============================] - 28s 114ms/step - loss: 1.2763 - accuracy: 0.4783
Epoch 2/10
114/114 [==============================] - 13s 111ms/step - loss: 0.7897 - accuracy: 0.6971
Epoch 3/10
114/114 [==============================] - 13s 111ms/step - loss: 0.6284 - accuracy: 0.7749
Epoch 4/10
114/114 [==============================] - 13s 110ms/step - loss: 0.5250 - accuracy: 0.8134
Epoch 5/10
114/114 [==============================] - 13s 111ms/step - loss: 0.4765 - accuracy: 0.8318
Epoch 6/10
114/114 [==============================] - 13s 111ms/step - loss: 0.4293 - accuracy: 0.8571
Epoch 7/10
114/114 [==============================] - 13s 111ms/step - loss: 0.4012 - accuracy: 0.8612
Epoch 8/10
114/114 [==============================] - 13s 110ms/step - loss: 0.3887 - accuracy: 0.8656
Epoch 9/10
114/114 [==============================] - 13s 111ms/step - loss: 0.3647 - accuracy: 0.8785
Epoch 10/10
114/114 [==============================] - 13s 110ms/step - l

In [16]:
# faccio un test per verificare che il modello sia funzionante
preprocessed_image = prepare_image('/content/flower_photos/roses/14494590921_3bb1dc7b88_n.jpg')
predictions_flower = finetune_model.predict(preprocessed_image) 
print(predictions_flower[0][0]*100)    
print(predictions_flower[0][1]*100) 
print(predictions_flower[0][2]*100) 
print(predictions_flower[0][3]*100) 
print(predictions_flower[0][4]*100) 

8.574289722673711e-05
1.1819594192274963e-06
99.01312589645386
8.468676355732896e-05
0.9867113083600998


##Conversione del Modello
h5 --> tflite --> kmodel

###h5 --> tflite

In [17]:
#converto da keras a tflite
!tflite_convert  --output_file=/content/model.tflite --keras_model_file=/content/my_model.h5

2022-09-08 09:56:34.226802: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-09-08 09:56:37.595639: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
INFO:tensorflow:Assets written to: /tmp/tmpbrs8dsw7/assets
I0908 09:56:41.545412 139825850173312 builder_impl.py:780] Assets written to: /tmp/tmpbrs8dsw7/assets
I0908 09:56:46.328571 139825850173312 lite.py:998] Using new converter: If you encounter a problem please file a bug. You can opt-out by setting experimental_new_converter=False
2022-09-08 09:56:46.390618: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-09-08 09:56:46.390671: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


### tflite --> kmodel

Preparo alcuni dati di esempio per il test ncc

In [18]:
%%bash
cd /content

mkdir /content/test_photos/
mkdir /content/test_photos/daisy
mkdir /content/test_photos/dandelion
mkdir /content/test_photos/roses
mkdir /content/test_photos/sunflowers
mkdir /content/test_photos/tulips


In [19]:
import os
import shutil


def recursive_copy(src, dest):
    """
    Copy each file from src dir to dest dir, including sub-directories.
    """
    
    #copio solo 10 file
    count=0
    for item in os.listdir(src):
        file_path = os.path.join(src, item)

        if os.path.isfile(file_path):
            shutil.copy(file_path, dest)

        
        elif os.path.isdir(file_path):
            new_dest = os.path.join(dest, item)
            os.mkdir(new_dest)
            recursive_copy(file_path, new_dest)
        if count == 10:
          break
        count+=1

recursive_copy("/content/flower_photos/daisy", "/content/test_photos/daisy")
recursive_copy("/content/flower_photos/dandelion", "/content/test_photos/dandelion")
recursive_copy("/content/flower_photos/roses", "/content/test_photos/roses")
recursive_copy("/content/flower_photos/sunflowers", "/content/test_photos/sunflowers")
recursive_copy("/content/flower_photos/tulips", "/content/test_photos/tulips")



In [20]:
#conversione tflite in formato kmodel
%cd /content/Maix_Toolbox
!./ncc/ncc -i tflite -o k210model --dataset /content/test_photos /content/model.tflite /content/model.kmodel

/content/Maix_Toolbox
0: InputLayer -> 1x3x224x224
1: K210Conv2d 1x3x224x224 -> 1x24x112x112
2: K210Conv2d 1x24x112x112 -> 1x24x112x112
3: K210Conv2d 1x24x112x112 -> 1x48x112x112
4: K210Conv2d 1x48x112x112 -> 1x48x112x112
5: K210Conv2d 1x48x112x112 -> 1x96x56x56
6: K210Conv2d 1x96x56x56 -> 1x96x56x56
7: K210Conv2d 1x96x56x56 -> 1x96x56x56
8: K210Conv2d 1x96x56x56 -> 1x96x56x56
9: K210Conv2d 1x96x56x56 -> 1x192x28x28
10: K210Conv2d 1x192x28x28 -> 1x192x28x28
11: K210Conv2d 1x192x28x28 -> 1x192x28x28
12: K210Conv2d 1x192x28x28 -> 1x192x28x28
13: K210Conv2d 1x192x28x28 -> 1x384x14x14
14: K210Conv2d 1x384x14x14 -> 1x384x14x14
15: K210Conv2d 1x384x14x14 -> 1x384x14x14
16: K210Conv2d 1x384x14x14 -> 1x384x14x14
17: K210Conv2d 1x384x14x14 -> 1x384x14x14
18: K210Conv2d 1x384x14x14 -> 1x384x14x14
19: K210Conv2d 1x384x14x14 -> 1x384x14x14
20: K210Conv2d 1x384x14x14 -> 1x384x14x14
21: K210Conv2d 1x384x14x14 -> 1x384x14x14
22: K210Conv2d 1x384x14x14 -> 1x384x14x14
23: K210Conv2d 1x384x14x14 -> 1x38